In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from scipy.ndimage import imread
import os
%matplotlib inline

## 1. Load the images and convert to grayscale for faster learning

In [60]:
os.chdir('/Users/agatagramatyka/Files/Jedha_maitrise/Week_5/train-walk')
listfiles = os.listdir()
walk_train = [plt.imread(a) for a in listfiles] # load and read the images into a list
walktrain_BW = [np.mean(a[:,:,:-1], axis=2) for a in walk_train] # convert the images into grayscale
walktrain_BW_arr = np.array(walktrain_BW) 
shape = walktrain_BW_arr.shape
walktrain_BW_arr = np.reshape(walktrain_BW_arr, (shape[0],1,shape[1],shape[2])) 
walktrain_y = np.zeros(shape = walktrain_BW_arr.shape[0]) # 0 for walk, 1 for run

In [61]:
os.chdir('/Users/agatagramatyka/Files/Jedha_maitrise/Week_5/train-run')
listfiles = os.listdir()
run_train = [plt.imread(a) for a in listfiles]
runtrain_BW = [np.mean(a[:,:,:-1], axis=2) for a in run_train]
runtrain_BW_arr = np.array(runtrain_BW)
shape = runtrain_BW_arr.shape
runtrain_BW_arr = np.reshape(runtrain_BW_arr, (shape[0],1,shape[1],shape[2]))
runtrain_y = np.zeros(shape = runtrain_BW_arr.shape[0]) + 1 # 0 for walk, 1 for run

In [62]:
os.chdir('/Users/agatagramatyka/Files/Jedha_maitrise/Week_5/test-run')
listfiles = os.listdir()
run_test = [plt.imread(a) for a in listfiles]
runtest_BW = [np.mean(a[:,:,:-1], axis=2) for a in run_test]
runtest_BW_arr = np.array(runtest_BW)
shape = runtest_BW_arr.shape
runtest_BW_arr = np.reshape(runtest_BW_arr, (shape[0],1,shape[1],shape[2]))
runtest_y = np.zeros(shape = runtest_BW_arr.shape[0]) + 1 # 0 for walk, 1 for run

In [63]:
os.chdir('/Users/agatagramatyka/Files/Jedha_maitrise/Week_5/test-walk')
listfiles = os.listdir()
walk_test = [plt.imread(a) for a in listfiles]
walktest_BW = [np.mean(a[:,:,:-1], axis=2) for a in walk_test]
walktest_BW_arr = np.array(walktest_BW)
shape = walktest_BW_arr.shape
walktest_BW_arr = np.reshape(walktest_BW_arr, (shape[0],1,shape[1],shape[2]))
walktest_y = np.zeros(shape = walktest_BW_arr.shape[0]) # 0 for walk, 1 for run

In [64]:
X_train = np.concatenate((walktrain_BW_arr, runtrain_BW_arr))
y_train = np.concatenate((walktrain_y, runtrain_y))
X_test = np.concatenate((walktest_BW_arr, runtest_BW_arr))
y_test = np.concatenate((walktest_y, runtest_y))

In [65]:
y_train.shape

(600,)

## 2. Build and apply a CNN model with Keras 

In [52]:
import keras as k

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [66]:
# Convert target variables to categorical
y_train = k.utils.to_categorical(y_train)
y_test = k.utils.to_categorical(y_test)

In [54]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

In [67]:
num_classes = y_test.shape[1]

In [68]:
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(1, 224, 224), activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [69]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=None, verbose=2)

Train on 600 samples, validate on 141 samples
Epoch 1/10
 - 171s - loss: 1.4305 - acc: 0.5133 - val_loss: 0.6922 - val_acc: 0.5816
Epoch 2/10
 - 202s - loss: 0.6932 - acc: 0.4983 - val_loss: 0.6921 - val_acc: 0.5816
Epoch 3/10
 - 233s - loss: 0.6932 - acc: 0.4983 - val_loss: 0.6920 - val_acc: 0.5816
Epoch 4/10
 - 177s - loss: 0.6932 - acc: 0.4983 - val_loss: 0.6921 - val_acc: 0.5816
Epoch 5/10
 - 153s - loss: 0.6932 - acc: 0.4983 - val_loss: 0.6922 - val_acc: 0.5816
Epoch 6/10
 - 157s - loss: 0.6932 - acc: 0.4983 - val_loss: 0.6922 - val_acc: 0.5816
Epoch 7/10
 - 155s - loss: 0.6932 - acc: 0.4983 - val_loss: 0.6923 - val_acc: 0.5816
Epoch 8/10
 - 478s - loss: 0.6932 - acc: 0.4983 - val_loss: 0.6923 - val_acc: 0.5816
Epoch 9/10
 - 177s - loss: 0.6932 - acc: 0.4983 - val_loss: 0.6924 - val_acc: 0.5816
Epoch 10/10
 - 196s - loss: 0.6932 - acc: 0.4983 - val_loss: 0.6926 - val_acc: 0.5816


In [70]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

CNN Error: 41.84%
